In [1]:
import os
import together
from typing import Any
from langchain.llms.base import LLM
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

from langchain.document_loaders import PyPDFDirectoryLoader

**NO TOCAR!**

In [2]:
os.environ["TOGETHER_API_KEY"] = "f8935229473a0d8a3f4709a9ef32533fe365c0cb215ba8c41413b5ca53a5c767"

class TogetherLLM(LLM):
    model: str = "togethercomputer/llama-2-7b-chat"
    temperature: float = 0.1
    max_tokens: int = 1024
    together_api_key: str = os.environ.get("TOGETHER_API_KEY")
    
    class Config:
        extra = 'forbid'
    @property
    def _llm_type(self) -> str:
        return "together"
    def _call(self, prompt: str, **kwargs: Any) -> str:
        if not self.together_api_key:
            raise ValueError("API key is not set.")
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                        model=self.model,
                                        max_tokens=self.max_tokens,
                                        temperature=self.temperature)
        print(f"modelo: {self.model}")
        if 'choices' in output:
            return output['choices'][0]['text']
        else:
            raise KeyError("The key 'choices' is not in the response.")

# Funcion para traduccion a ingles
from googletrans import Translator
import googletrans
translator = Translator()

def traducir(texto_original):
    try :
        origen = translator.detect(texto_original).lang
        # Si el texto esta en otro idioma que no sea ingles lo traduce
        if origen != "en":
            #print(f'Traduccion de {origen} a en')
            traduccion = translator.translate(texto_original, dest="en", src=origen).text
            return traduccion
        # En caso contrario devuelve el texto original ya que esta en ingles
        else:
            return texto_original
    except:
        return texto_original
    
# Funcion para traduccion a otro idioma
def traducir_ingles(texto_ingles, idioma_destino):
    for abreviatura, nombre_idioma in googletrans.LANGUAGES.items():
        if nombre_idioma == idioma_destino:
            destino = abreviatura
    traduccion = translator.translate(texto_ingles, dest=destino, src="en").text
    return traduccion

def dividir_texto(texto, max_caracteres):
    textos_divididos = []
    texto_actual = ''
    caracteres_actuales = 0

    oraciones = texto.split('.')

    for oracion in oraciones:
        caracteres_actuales += len(oracion) + 1

        if caracteres_actuales <= max_caracteres:
            texto_actual += oracion + '.'
        else:
            textos_divididos.append(texto_actual.strip())
            texto_actual = oracion + '.'
            caracteres_actuales = len(oracion) + 1

    textos_divididos.append(texto_actual.strip())

    return textos_divididos

def get_prompt(instruction, new_system_prompt ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

sys_prompt = """You are a helpful, respectful and honest helper. 
Always answer in the most helpful way possible using the contextual text provided, do not add information which is not in that context. Your answers should only answer the question once and should not contain any text after the answer.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. 

If the question is not directly related to the provided context, politely inform the user that the question is outside the context scope and cannot be answered accurately.

Ensure that your answers are clear and concise, avoiding ambiguity or vague responses."""

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

In [3]:
def create_embeddings(pdf):
    pdf_reader = PdfReader(pdf)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    # Traducir text a ingles
    # max_caracteres = 14000
    # textos_divididos = dividir_texto(text, max_caracteres)
    # text = ""
    # for textSec in textos_divididos:
    #     text += textSec

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
        )
    
    chunks = text_splitter.split_text(text)

    # print(f"tamano de chunks: {len(chunks)}")
    # for i in range(len(chunks)):
    #     print(f"chunk {i}: {len(chunks[i])}")

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
    knowledge_base = FAISS.from_texts(chunks, embeddings)
    return knowledge_base


prompt_template = get_prompt(instruction, sys_prompt)
llama_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": llama_prompt}

def modelo_llm(modelo):
    return TogetherLLM(
        model= modelo,
    )


In [4]:
def conversation_chat(query,chain):
    #result =chain({"question":query, "chat_history": history})
    user_question_eng = traducir(query)
    llm_response = chain.invoke(user_question_eng)
    llm_response = traducir_ingles(llm_response['result'], "spanish")
    return llm_response

In [5]:
pdf_obj = "./historia.pdf"
knowledge_base = create_embeddings(pdf_obj)

retriever = knowledge_base.as_retriever(search_kwargs={"k": 3})
modelo = "togethercomputer/llama-2-7b-chat"
llm = modelo_llm(modelo)
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    chain_type_kwargs=chain_type_kwargs)

user_question = "De que trata el documento?"
conversation_chat(user_question, qa_chain )

/home/dante/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/dante/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/tmp/ipykernel_20913/3817744212.py:18: DeprecationWarning: Call to deprecated function create.
  output = together.Complete.create(prompt,
/home/dante/.local/lib/python3.10/site-packages/together/legacy/complete.py:23: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)


modelo: togethercomputer/llama-2-7b-chat


'El documento trata sobre la creación de un Tribunal Militar Internacional para procesar y castigar a los criminales de guerra, en particular a los responsables de crímenes de lesa humanidad, durante la Segunda Guerra Mundial. El documento describe la composición y funcionamiento del tribunal, así como los crímenes que procesará.'